<a href="https://colab.research.google.com/github/MariusLotz/myTransformerExp/blob/main/myTransformer_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<center><h1>The Annotated Transformer</h1> </center>


<center>
<p><a href="https://arxiv.org/abs/1706.03762">Attention is All You Need
</a></p>
</center>

<img src="https://github.com/harvardnlp/annotated-transformer/blob/master/images/aiayn.png?raw=1" width="70%"/>

* adapted minimal version 2023: Marius Lotz
* *[Original](https://nlp.seas.harvard.edu/2018/04/03/attention.html):
   [Sasha Rush](http://rush-nlp.com/).*




# Installing and importing modules:



In [1]:
"""Alle nötigen Downloads:"""
#!pip install -q pandas==1.3.5
#!pip install -q torch==1.11.0+cu113
#!pip install -q torchdata==0.3.0
#!pip install -q torchtext==0.12
#!pip install -q spacy==3.2
#!pip install -q altair==4.1
#!pip install -q GPUtil
#!pip install -q wandb
#!pip install wget


'Alle nötigen Downloads:'

In [2]:
""" Nötige Downloads für Google-Colab: """
!pip install -q torchdata==0.3.0 torchtext==0.12 spacy==3.2 altair GPUtil wget # Module
!python -m spacy download de_core_news_sm # spacy, deutsches Vokabular
!python -m spacy download en_core_web_sm # spacey, englisches Vokabular



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.4/10.4 MB 53.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 73.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 659.5/659.5 kB 58.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 15.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.5.0 requires spacy<3.6.0,>=3.5.0, but you have spacy 3.2.0 which is incompatible.
inflect 6.0.4 requires pydantic>=1.9.1, but you have pydantic 1.8.2 which is incompatible.
torchaudio 2.0.2+cu118 requires torch==2.0.1, but you have torch 1.11.

In [23]:
"""Nötige Imports und Helper Funktionen:"""
import os
import wget
from os.path import exists
import torch
import torch.nn as nn
from torch.nn.functional import log_softmax, pad
import math
import copy
import time
from torch.optim.lr_scheduler import LambdaLR
import pandas as pd
import altair as alt
from torchtext.data.functional import to_map_style_dataset
from torch.utils.data import DataLoader
from torchtext.vocab import build_vocab_from_iterator
import torchtext.datasets as datasets
import spacy
import GPUtil
import warnings
from torch.utils.data.distributed import DistributedSampler
import torch.distributed as dist
import torch.multiprocessing as mp
from torch.nn.parallel import DistributedDataParallel as DDP


# Helpers:
warnings.filterwarnings("ignore")
RUN_EXAMPLES = True
def show_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        return fn(*args)

def execute_example(fn, args=[]):
    if __name__ == "__main__" and RUN_EXAMPLES:
        fn(*args)

class DummyOptimizer(torch.optim.Optimizer):
    def __init__(self):
        self.param_groups = [{"lr": 0}]
        None

    def step(self):
        None

    def zero_grad(self, set_to_none=False):
        None

class DummyScheduler:
    def step(self):
        None

# Part 1: Aufbau des Transformers

<div>
Der Transformer ist ein neuronales Netzwerkmodell, welchen in dem Paper [Attention Is All You Need](https://arxiv.org/abs/1706.03762) im Jahr 2017 vorgestellt wurde. Es verwendet eine Attention-Mechanismus-basierte Architektur, um effektiv und parallel Informationen über die Eingabesequenz zu erfassen. Die Hauptkomponenten und Transformationsschritte eines Transformers lauten:

1. Eingabeembeddings:
Die Eingabesequenz besteht aus one-hot Vektoren, welche in Eingabeembeddings tranformiert werden.
Hierbei findet eine Dimensionsreduktion statt.

2. Positional Encoding:
Da der Transformer keine Rekurrenz oder Faltung verwendet, wird eine Positionscodierung hinzugefügt (addiert), um die Positionsinformationen der Tokens in der Sequenz zu erfassen.


</div>
<div>
<img src="https://github.com/harvardnlp/annotated-transformer/blob/master/images/ModalNet-21.png?raw=1" align="left" margin-right="20px">
</div>
<div>

3. Encoder (links):
Der Encoder besteht aus mehreren (N) identischen Schichten, die als Encoder-Layer bezeichnet werden.
Jeder Encoder-Layer enthält zwei Hauptsublayer:
Multi-Head Attention und Feed-Forward Network (FFN).

4. Multi-Head Attention:
Die Multi-Head Attention ermöglicht dem Transformer, Beziehungen zwischen den Tokens in der Eingabesequenz zu modellieren.
Sie besteht aus mehreren Attention-Heads, die parallel arbeiten und unterschiedliche Repräsentationen der Eingabesequenz lernen.
Jeder Attention-Head berechnet gewichtete Aufmerksamkeitsvektoren, um die Bedeutung jedes Tokens in Bezug auf alle anderen Token zu bestimmen.

5. Feed-Forward Network (FFN):
Das FFN wird auf jedes Token in der Sequenz angewendet. Es ermöglicht dem Transformer, nichtlineare Transformationen auf die Repräsentation der Tokens durchzuführen.

6. Residual Connection und Layer Normalization:
Zwischen den Sublayern des Encoders werden Residual Connections und Layer Normalization angewendet, um stabiles und schnelles Lernen zu ermöglichen.
Die Residual Connection fügt die Eingabe des Sublayers zu seiner Ausgabe hinzu, um Informationen von der Eingabe zu erhalten.
Die Layer Normalization normalisiert die Ausgabe des Sublayers, um die Gradientenpropagation zu verbessern.

7. Decoder (rechts): Der Decoder hat eine ähnliche Architektur wie der Encoder, besteht jedoch aus Decoder-Layern.
Jeder Decoder-Layer enthält zusätzlich zur Multi-Head Attention und dem FFN eine weitere Multi-Head Attention-Schicht, die den Encoder-Output als Quelle verwendet.
Die zusätzliche Multi-Head Attention ermöglicht dem Decoder, Informationen über den Kontext der Eingabesequenz zu berücksichtigen.

8. Generator (rechts oben):
Der Generator ist eine lineare Schicht, die die Ausgabe des Decoders in die endgültige Ausgabe umwandelt (PseudoInverse der Projektion).
Sie führt eine Klassifikation oder eine Wahrscheinlichkeitsverteilung über das Vokabular durch, um das nächste Token in der Sequenz zu generieren.

</div>

## Attention-Funktion

Die Eingabe $x$ des Transformers wird durch drei verschiedene lineare Transformationen $W^q, W^k, W^v$ weiterverarbeitet. Danach erhält man die Query-, Key- und Value-Vektoren mit denen man dann die kontextbezogenen Embeddings bestimmen kann, welche durch

$$
A_j = ∑_{i=1}^T \frac{exp(a_{i,j})}{∑_{j=1}^T exp(a_{i,j})} \cdot v_i
$$

bestimmt sind. Hierbei ist $v_i = x_i W^v$ und

$$
a_{i,j} = <q_i, k_j> = <x_i W^q, x_j W^k>.
$$


Diese Transformationen ermöglichen es dem Modell, die Beziehungen zwischen den Wörtern im Eingabesatz zu erfassen. Da Matrixmultiplikation schneller möglich ist als alle $A_j$ einzeln zu berechnen, werden alle alle Kontextbezogenen Embeddings mit Hilfe der unteren Matrix Multiplikation zusammen berechnet.


$$
   A= \begin{pmatrix}
A_1 \\
... \\
A_T \\
\end{pmatrix} = \mathrm{Attention}(Q, K, V) = \mathrm{softmax}(\frac{QK^T}{\sqrt{d_k}})V
$$


In [4]:
def attention(query, key, value, mask=None, dropout=None):
    """Berechne die 'Scaled Dot Product Attention'"""
    d_k = query.size(-1) # repräsentiert die Dimension der keys und queries
    scores = torch.matmul(query, key.transpose(-2, -1)) / math.sqrt(d_k) # Teil in der softmax Funktion
    if mask is not None:
        scores = scores.masked_fill(mask == 0, -1e9) # Durch eine Maske geblockte Einträge erhalten ein sehr hohen negativen Attention Score
    p_attn = scores.softmax(dim=-1)
    if dropout is not None:
        p_attn = dropout(p_attn) # Dropout beim Training
    return torch.matmul(p_attn, value), p_attn


![](https://github.com/MariusLotz/myTransformerExp/raw/main/images/attention-raschka.png)

<br>
Im obigen Bild, wird nochmal die Berechnung auf Vektorebene veranschaulicht, hier für das kontextbezogene Embedding $A_2$.
<br> <br>

<hr color="red">

<br>
Im unteren Bild, wird die Berechnung auf Matrixebene veranschaulicht. Man sieht die einzelnen Schritte für die Berechnung
der Attention Matrix A.  
<br>
<br>

![](https://github.com/MariusLotz/myTransformerExp/raw/main//images/attention3-raschka.png)



## I/O Embedding & Positional Encoding
<div>
<img src="https://github.com/MariusLotz/myTransformerExp/raw/main/images/Embedding.png" align="right" margin-right="20px">
</div>
<div>

Jeder Token liegt vor dem Embedding als one-hot Vektor vor. Somit ist der Raum aller Token i.d.R extrem hochdimensional und muss daher auf einen kleineren Raum projeziert werden. Dieser Schritt wird Embedding genannt. Hierbei wird jeder one-hot Vektor durch eine erlernbare Embedding-Matrix auf seine stetige Repräsentation projeziert. Die Embeddings erfassen die semantische Bedeutung der Token und liefern eine informativere Darstellung für nachgelagerte Aufgaben. Im Paper sind alle Embeddings von der Dimension $d_{model} =512$.

</div>




In [5]:
class Embeddings(nn.Module):
    """Projektion der Sequenz in die Dimension d_model"""
    def __init__(self, d_model, vocab): # d_model=512, vocab=Anzahl aller Token
        super(Embeddings, self).__init__()
        self.lut = nn.Embedding(vocab, d_model)
        # self.lut ist eine Instanz der nn.Embedding-Klasse, die als Embedding-Schicht fungiert
        # Die Schicht projiziert die Token der Sequenz von einem one-hot kodierten Vektor in einen kontinuierlichen Embedding-Vektor im d_model-dimensionalen Raum
        self.d_model = d_model

    def forward(self, x):
        """Forwardpass der Embedding Schicht"""
        return self.lut(x) * math.sqrt(self.d_model) # Varianzreduktion / Normalisierung des Embeddings

Um ebenfalls die Reihenfolge der Wörter in den Sequenzen zu erfassen wird an die Embeddings noch ein kleiner Teil addiert, welcher gegeben ist durch: <br>
$$PE_{(pos,2i)} = \sin(pos / 10000^{2i/d_{\text{model}}})$$

$$PE_{(pos,2i+1)} = \cos(pos / 10000^{2i/d_{\text{model}}})$$
<br>
Hierbei ist $pos \in \{1, ..., T\}$ die Position des Tokens in der Sequenz und $i \in \{1, ..., d_{model}\}$  ist die Dimension der Embeddings, hier also $d_{model} = 512$.

In [6]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout) #  Dropout-Schicht

        # Compute the positional encodings once in log space.
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len).unsqueeze(1)
        # Die Positionsencodings werden einmalig in logarithmischen Raum berechnet:
        div_term = torch.exp(
            torch.arange(0, d_model, 2) * -(math.log(10000.0) / d_model)
        )
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer("pe", pe)

    def forward(self, x):
        """Forwardpass der PositionalEnconding Schicht"""
        # Die Positionsencodings werden addiert:
        x = x + self.pe[:, : x.size(1)].requires_grad_(False)
        return self.dropout(x)

###Add & Norm  
<div>
<img src="https://github.com/MariusLotz/myTransformerExp/raw/main/images/SkipLayer.png" align="left" padding-right="40px">
</div>
<div>

Der Input x des Sublayers wird zu

<br>
$$LayerNorm(x+Sublayer(x))$$ <br>

transformiert, wobei Sublayer die Funktion des Sublayers selbst ist.


Die möglichen Sublayer hier sind Multi-Head Attention Layer und Feed-Forward Layer.
</div>

In [7]:
class LayerNorm(nn.Module):
    """Implementierung der Layer Normalisierung"""
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        """Forwardpass der LayerNorm Schicht"""
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2


class SublayerConnection(nn.Module):
    """Implementierung der Skip-Connection Verbindung"""

    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        """Forward Pass der Skip-Connection"""
        return x + self.dropout(sublayer(self.norm(x))) # wieder mit Dropout-Schicht

## Multi-Head Attention Layer

Das Multi-Head Attention Layer zerlegt die Eingabe in mehrere Teilsequenzen gleicher Länge. Im Paper sind es 8 Köpfe, dh.
jede Teilsequenz hat die Länge $d_v' = \frac{d_v}{h} = \frac{512}{8} = 64 $. Für jede Teilsequenz wird die Attention Matrix berechnet.
Die "Aufmerksamkeitsköpfe" arbeiten parallel. Sie werden eingesetzt um verschiedene Darstellungen der Eingabe zu erlernen. Jeder Kopf führt eine eigenständige Attention-Berechnung durch.
Danach werden alle Attention Matrizen aneinandergehängt, sodass wir wieder die Ausgangsgröße $d_v$ erhalten ("Concat" in der Abbildung)
Die zusammengefügte Attention Matrix wird danach mit einer (erlernbaren) Matrix multipliziert ("Linear" in der Abbildung).
<div>
<img src="https://github.com/MariusLotz/myTransformerExp/raw/main/images/attention4-raschka.png" align="left" margin-right="20px">
</div>

In [8]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
        "Take in model size and number of heads."
        super(MultiHeadedAttention, self).__init__()
        assert d_model % h == 0 # Dimension d_model muss durch h teilbar sein
        # Es wird angenommen d_v ist auch immer gleich d_k
        self.d_k = d_model // h
        self.h = h # Anzahl Köpfe
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

    def forward(self, query, key, value, mask=None):
        """Forwardpass des Multi-Head Attention Layers"""
        if mask is not None:
            # Maske wird auf alle Köpfe angewandt
            mask = mask.unsqueeze(1)
        nbatches = query.size(0)

        # 1) Berechnung aller querys, keys und values:
        query, key, value = [
            lin(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for lin, x in zip(self.linears, (query, key, value))
        ]

        # 2) Berechnung aller Attention Matrizen:
        x, self.attn = attention(
            query, key, value, mask=mask, dropout=self.dropout
        )

        # 3) Aneinanderhängen aller Attention Matrizen:
        x = (
            x.transpose(1, 2)
            .contiguous()
            .view(nbatches, -1, self.h * self.d_k)
        )
        # Speicher freimachen:
        del query
        del key
        del value
        return self.linears[-1](x) # Attention Matrix wird linear abgebildet

## Feed-Forward Layer
 Das Feed-Forward Layer wird auf jede Position der Eingabesequenz (Token) einzeln angewandt. Hierbei wird jeder Token x zu

<br>
$$ FFN(x) = max(0, xW_1)W_2 + b_2 $$
<br>

transformiert.
Die Dimension eines jeden Token im Paper ist $d_{model}=512$. <br>
Für jedes der $N=6$ Layer werden unterschiedliche Gewichte und Bias berechnet.

In [9]:
class PositionwiseFeedForward(nn.Module):
    """Implementierung des Feed-Forward NN"""

    def __init__(self, d_model, d_ff, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        """Forwardpass des NN"""
        return self.w_2(self.dropout(self.w_1(x).relu())) # wieder mit Dropout

## Aufbau des Transformers
Der Transformer besteht neben den Embedding Schichten aus dem
Encoder, Decoder und Generator, welche hier definiert werden.

In [10]:
class EncoderDecoder(nn.Module): # Klasse erbt von nn.Module
    """Vanilla Encoder-Decoder Architektur"""

    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder # Encoder Teil
        self.decoder = decoder # Decoder Teil
        self.src_embed = src_embed # Input Encoder (Embedding)
        self.tgt_embed = tgt_embed # Input Decoder (Embedding)
        self.generator = generator # Output Decoder

    def forward(self, src, tgt, src_mask, tgt_mask):
        """Forward Pass des Transformers"""
        return self.decode(self.encode(src, src_mask), src_mask, tgt, tgt_mask)

    def encode(self, src, src_mask):
        """Forwardpass encode"""
        return self.encoder(self.src_embed(src), src_mask)

    def decode(self, memory, src_mask, tgt, tgt_mask):
        """Forwardpass decode"""
        return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)


class Generator(nn.Module):
    """'Pseudoinverse' der Projektion"""

    def __init__(self, d_model, vocab):
        super(Generator, self).__init__()
        self.proj = nn.Linear(d_model, vocab)

    def forward(self, x):
        return log_softmax(self.proj(x), dim=-1) # Wahrscheinlichkeitsmaß

def clones(module, N):
    """Erzeugt N identische Klone des Eingabe Layers=module"""
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

### Encoder


In [11]:
class Encoder(nn.Module):
    """Encoder ist ein Stack von N Layers"""

    def __init__(self, layer, N):
        super(Encoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, mask):
        """Input wird durch alle Layer geschickt"""
        for layer in self.layers:
            x = layer(x, mask)
        return self.norm(x)


class EncoderLayer(nn.Module):
    """Encoder besteht aus dem Attention Layer und dem FFN, sowie einem Dropout Layer beim Lernen"""
    def __init__(self, size, self_attn, feed_forward, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = self_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 2)
        self.size = size

    def forward(self, x, mask):
        """Forwardpass des Encoder Layers"""
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
        return self.sublayer[1](x, self.feed_forward)

### Decoder


In [12]:
class Decoder(nn.Module):
    """Decoder ist ein Stack von N Layers mit Maske."""
    def __init__(self, layer, N):
        super(Decoder, self).__init__()
        self.layers = clones(layer, N)
        self.norm = LayerNorm(layer.size)

    def forward(self, x, memory, src_mask, tgt_mask):
        """Input wird durch alle Layer geschickt"""
        for layer in self.layers:
            x = layer(x, memory, src_mask, tgt_mask)
        return self.norm(x)


class DecoderLayer(nn.Module):
    """Decoder besteht aus dem Self-Attention Layer, dem Source Attention Layer und dem FFN, sowie einem Dropout Layer beim Lernen"""
    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
        super(DecoderLayer, self).__init__()
        self.size = size
        self.self_attn = self_attn
        self.src_attn = src_attn
        self.feed_forward = feed_forward
        self.sublayer = clones(SublayerConnection(size, dropout), 3)

    def forward(self, x, memory, src_mask, tgt_mask):
        "Forwardpass des Decoder Laayers"""
        m = memory
        x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
        x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
        return self.sublayer[2](x, self.feed_forward)


def subsequent_mask(size):
    """Maskiere nachfolgende Positionen, damit der Tranformer beim Training nicht schummeln kann"""
    attn_shape = (1, size, size)
    subsequent_mask = torch.triu(torch.ones(attn_shape), diagonal=1).type(
        torch.uint8
    )
    return subsequent_mask == 0


def example_mask():
    """Beispiel für eine subsequent_mask"""
    LS_data = pd.concat(
        [
            pd.DataFrame(
                {
                    "Subsequent Mask": subsequent_mask(20)[0][x, y].flatten(),
                    "Window": y,
                    "Masking": x,
                }
            )
            for y in range(20)
            for x in range(20)
        ]
    )

    return (
        alt.Chart(LS_data)
        .mark_rect()
        .properties(height=250, width=250)
        .encode(
            alt.X("Window:O"),
            alt.Y("Masking:O"),
            alt.Color("Subsequent Mask:Q", scale=alt.Scale(scheme="viridis")),
        )
        .interactive()
    )


print("Beispielmaske")
show_example(example_mask)

Beispielmaske


alt.Chart(...)

### Modell erstellen

In [13]:
def make_model(src_vocab, tgt_vocab, N=6,
               d_model=512, d_ff=2048, h=8, dropout=0.1):
    """
    src_vocab = Anzahl der Token im Eingabetext
    tgt_vocab = Anzahl der Token im Ausgabetext
    N = Anzahl Schichten Encoder / Decoder
    d_model = Dimension des Embeddings
    d_ff = Dimension der Feedforward Schicht
    h = Anzahl der Attention Heads
    dropout = Dropout Rate für alle Schichten
    """
    c = copy.deepcopy
    attn = MultiHeadedAttention(h, d_model) # Attention Funktion
    ff = PositionwiseFeedForward(d_model, d_ff, dropout) # Feed Forward NN Funktion
    position = PositionalEncoding(d_model, dropout)
    model = EncoderDecoder( # Definition des Tranformers
        Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
        Decoder(DecoderLayer(d_model, c(attn), c(attn),
                             c(ff), dropout), N),
        nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
        nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
        Generator(d_model, tgt_vocab))

    # Bestimmte Initalisierung der Parameter:
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform(p)
    return model

# Part 2: Training des Transformers

## Definition des Trainingsmodells
Es werden einige der Werkzeuge eingeführt, die benötigt werden, um ein Standard-Encoder-Decoder-Modell zu trainieren.
Diese umfassen die Definition eines Batches, einer Verlustfunktion, einer Lernrate sowie die Funktion TrainState und Label Smoothing.

In [14]:
class Batch:
    """Organisation und Verarbeitung einer Datencharge für das Training im Transformer-Modell"""

    def __init__(self, src, tgt=None, pad=2):  # 2 = <blank>
        self.src = src
        self.src_mask = (src != pad).unsqueeze(-2) # Maske für die Quellsequenz
        if tgt is not None:
            # speichert die Zielsequenz ohne das letzte Token
            # Dadurch wird eine Verschiebung um einen Schritt erreicht, um das Modell zu trainieren, das nächste Token vorherzusagen:
            self.tgt = tgt[:, :-1]
            # speichert die Zielsequenz ohne das erste Token
            # Dies dient dazu, das Modell zu trainieren, das jeweils nächste Token vorherzusagen.
            self.tgt_y = tgt[:, 1:]
            self.tgt_mask = self.make_std_mask(self.tgt, pad) # Maske für die Zielsequenz, die sowohl Padding-Elemente als auch zukünftige Wörter blockiert
            self.ntokens = (self.tgt_y != pad).data.sum()  # Anzahl der nicht-Padding-Token in der Zielsequenz

    @staticmethod
    def make_std_mask(tgt, pad):
        "Standardmaske die Padding und zukünftige Wörter blockt."
        tgt_mask = (tgt != pad).unsqueeze(-2)
        tgt_mask = tgt_mask & subsequent_mask(tgt.size(-1)).type_as(
            tgt_mask.data
        )
        return tgt_mask


class SimpleLossCompute:
    "Loss Funktion ."

    def __init__(self, generator, criterion):
        self.generator = generator # Ausgabe Generator bestimmt den Verlust beim Training
        self.criterion = criterion

    def __call__(self, x, y, norm):
        """Berechnung des Verlusts nach dem Kriterium (criterion)"""
        x = self.generator(x)
        sloss = (
            self.criterion(
                x.contiguous().view(-1, x.size(-1)), y.contiguous().view(-1)
            )
            / norm
        )
        return sloss.data * norm, sloss


class TrainState:
    """Klasse dient dazu den Fortschritt des Trainingsprozesses zu verfolgen."""

    step: int = 0  # Steps in the current epoch
    accum_step: int = 0  # Number of gradient accumulation steps
    samples: int = 0  # total # of examples used
    tokens: int = 0  # total # of tokens processed


def run_epoch(
    data_iter, # der Dateniterator
    model, # zu trainierendes Modell
    loss_compute, # Verlustfunktion
    optimizer, # Gradientenverfahren z.B. Adam
    scheduler, # der Lernratenplaner
    mode="train",
    accum_iter=1, # die Anzahl der Gradientenakkumulationsschritte
    train_state=TrainState(),
):
    """Eine Epoche Training"""
    start = time.time()
    total_tokens = 0
    total_loss = 0
    tokens = 0
    n_accum = 0
    for i, batch in enumerate(data_iter):
        out = model.forward(
            batch.src, batch.tgt, batch.src_mask, batch.tgt_mask
        )
        loss, loss_node = loss_compute(out, batch.tgt_y, batch.ntokens)
        # loss_node = loss_node / accum_iter
        if mode == "train" or mode == "train+log":
            loss_node.backward()
            # Trainingszustand wird aktualisiert
            train_state.step += 1
            train_state.samples += batch.src.shape[0]
            train_state.tokens += batch.ntokens
            if i % accum_iter == 0:
                optimizer.step()
                optimizer.zero_grad(set_to_none=True)
                n_accum += 1
                train_state.accum_step += 1
            scheduler.step()

        total_loss += loss # Gesamtverlust wird aktualisiert
        total_tokens += batch.ntokens # Gesamtanzahl Token werden aktualisiert
        tokens += batch.ntokens
        if i % 40 == 1 and (mode == "train" or mode == "train+log"):
            lr = optimizer.param_groups[0]["lr"]
            elapsed = time.time() - start
            print(
                (
                    "Epoch Step: %6d | Accumulation Step: %3d | Loss: %6.2f "
                    + "| Tokens / Sec: %7.1f | Learning Rate: %6.1e"
                )
                % (i, n_accum, loss / batch.ntokens, tokens / elapsed, lr)
            )
            start = time.time()
            tokens = 0
        del loss
        del loss_node
        # Rückgabe des durchschnittliche Verlust pro Token zusammen mit dem Trainingsstatus:
    return total_loss / total_tokens, train_state


def rate(step, model_size, factor, warmup):
    """Lernrate für einen bestimmten Schritt in der Trainingsphase eines Transformer-Modells"""
    if step == 0:
        step = 1 # wegen Formel unten (nicht durch 0 teilen)
    return factor * (
        model_size ** (-0.5) * min(step ** (-0.5), step * warmup ** (-1.5))
    )


class LabelSmoothing(nn.Module):
    """
    Label-Smoothing ersetzt den one-hot-kodierten Label-Vektor y_hot durch eine Mischung aus y_hot und einer gleichmäßigen Verteilung:
    y_ls = (1 - α) * y_hot + α / K
    """

    def __init__(self, size, padding_idx, smoothing=0.0):
        super(LabelSmoothing, self).__init__()
        self.criterion = nn.KLDivLoss(reduction="sum") # Verlustfunktion
        self.padding_idx = padding_idx
        self.confidence = 1.0 - smoothing
        self.smoothing = smoothing
        self.size = size
        self.true_dist = None

    def forward(self, x, target):
        """Forwardpass des LabelSmoothing"""
        assert x.size(1) == self.size
        true_dist = x.data.clone()
        true_dist.fill_(self.smoothing / (self.size - 2))
        true_dist.scatter_(1, target.data.unsqueeze(1), self.confidence)
        true_dist[:, self.padding_idx] = 0
        mask = torch.nonzero(target.data == self.padding_idx)
        if mask.dim() > 0:
            true_dist.index_fill_(0, mask.squeeze(), 0.0)
        self.true_dist = true_dist
        return self.criterion(x, true_dist.clone().detach())

## Laden des Trainingssets

In diesem Teil wird das Trainingsset und die Trainingbatches erstellt.



In [19]:
def load_tokenizers():
    """Lade das spacy Modul für das Tokenisieren"""
    try:
        spacy_de = spacy.load("de_core_news_sm")
    except IOError:
        os.system("python -m spacy download de_core_news_sm")
        spacy_de = spacy.load("de_core_news_sm")

    try:
        spacy_en = spacy.load("en_core_web_sm")
    except IOError:
        os.system("python -m spacy download en_core_web_sm")
        spacy_en = spacy.load("en_core_web_sm")

    return spacy_de, spacy_en


def tokenize(text, tokenizer):
    """Hilfsfunktion für den Tokenisierer"""
    return [tok.text for tok in tokenizer.tokenizer(text)]


def yield_tokens(data_iter, tokenizer, index):
    """Hilfsfunktion für den Tokenisierer"""
    for from_to_tuple in data_iter:
        yield tokenizer(from_to_tuple[index])


def build_vocabulary(spacy_de, spacy_en):
    """Tokenisiere das Vocabular"""
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    print("Building German Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_src = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_de, index=0),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    print("Building English Vocabulary ...")
    train, val, test = datasets.Multi30k(language_pair=("de", "en"))
    vocab_tgt = build_vocab_from_iterator(
        yield_tokens(train + val + test, tokenize_en, index=1),
        min_freq=2,
        specials=["<s>", "</s>", "<blank>", "<unk>"],
    )

    vocab_src.set_default_index(vocab_src["<unk>"])
    vocab_tgt.set_default_index(vocab_tgt["<unk>"])

    return vocab_src, vocab_tgt


# Vokabelm manuell runterladen anstatt zu erzeugen:
#url_pt = "https://github.com/MariusLotz/myTransformerExp/blob/main/vocab.pt"
#file_path = wget.download(url_pt)
#print("downloaded vocab is used")


def load_vocab(spacy_de, spacy_en):
    if not exists("vocab.pt"):
        vocab_src, vocab_tgt = build_vocabulary(spacy_de, spacy_en)
        torch.save((vocab_src, vocab_tgt), "vocab.pt")
    else:
        vocab_src, vocab_tgt = torch.load("vocab.pt")
    print("Finished.\nVocabulary sizes:")
    print(len(vocab_src))
    print(len(vocab_tgt))
    return vocab_src, vocab_tgt


def collate_batch(
    batch,
    src_pipeline,
    tgt_pipeline,
    src_vocab,
    tgt_vocab,
    device,
    max_padding=128,
    pad_id=2,
):
    """Batch Preprocessing: tokenization und padding->Ganzzahl-Tensoren"""
    bs_id = torch.tensor([0], device=device)  # <s> token id
    eos_id = torch.tensor([1], device=device)  # </s> token id
    src_list, tgt_list = [], []
    for (_src, _tgt) in batch:
        processed_src = torch.cat(
            [
                bs_id,
                torch.tensor(
                    src_vocab(src_pipeline(_src)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        processed_tgt = torch.cat(
            [
                bs_id,
                torch.tensor(
                    tgt_vocab(tgt_pipeline(_tgt)),
                    dtype=torch.int64,
                    device=device,
                ),
                eos_id,
            ],
            0,
        )
        src_list.append(
            # warning - overwrites values for negative values of padding - len
            pad(
                processed_src,
                (
                    0,
                    max_padding - len(processed_src),
                ),
                value=pad_id,
            )
        )
        tgt_list.append(
            pad(
                processed_tgt,
                (0, max_padding - len(processed_tgt)),
                value=pad_id,
            )
        )

    src = torch.stack(src_list)
    tgt = torch.stack(tgt_list)
    return (src, tgt)


def create_dataloaders(
    device,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    batch_size=12000,
    max_padding=128,
    is_distributed=True,
):
    # Erstelle Trainings- und Validierungsdatenlader mit Batch_size=12000:
    def tokenize_de(text):
        return tokenize(text, spacy_de)

    def tokenize_en(text):
        return tokenize(text, spacy_en)

    def collate_fn(batch):
        return collate_batch(
            batch,
            tokenize_de,
            tokenize_en,
            vocab_src,
            vocab_tgt,
            device,
            max_padding=max_padding,
            pad_id=vocab_src.get_stoi()["<blank>"],
        )

    train_iter, valid_iter, test_iter = datasets.Multi30k(
        language_pair=("de", "en")
    )

    train_iter_map = to_map_style_dataset(
        train_iter
    )  # DistributedSampler needs a dataset len()
    train_sampler = (
        DistributedSampler(train_iter_map) if is_distributed else None
    )
    valid_iter_map = to_map_style_dataset(valid_iter)
    valid_sampler = (
        DistributedSampler(valid_iter_map) if is_distributed else None
    )

    train_dataloader = DataLoader(
        train_iter_map,
        batch_size=batch_size,
        shuffle=(train_sampler is None),
        sampler=train_sampler,
        collate_fn=collate_fn,
    )
    valid_dataloader = DataLoader(
        valid_iter_map,
        batch_size=batch_size,
        shuffle=(valid_sampler is None),
        sampler=valid_sampler,
        collate_fn=collate_fn,
    )
    return train_dataloader, valid_dataloader

###Modell trainieren:

![](https://github.com/MariusLotz/myTransformerExp/raw/main/images/Translation.png)

Im Folgenden wird das Modell trainiert (bzw. geladen). Auskommentieren von (Laden) führt dazu, dass das Modell neu trainiert wird, was bis zu einer halben Stunde dauern kann.

In [20]:
# Im Moment nicht auskommentieren, sondern multi30k_model_final.pt manuell hochladen
# Laden (auskommentieren, wenn Modell neu berechnet werden soll):
# url_pt = "https://github.com/MariusLotz/myTransformerExp/blob/main/data/multi30k_model_final.pt"
# file_path = wget.download(url_pt)


# Globale Variablen
spacy_de, spacy_en = show_example(load_tokenizers)
vocab_src, vocab_tgt = show_example(load_vocab, args=[spacy_de, spacy_en])


def train_worker(
    gpu,
    ngpus_per_node,
    vocab_src,
    vocab_tgt,
    spacy_de,
    spacy_en,
    config,
    is_distributed=False,
):
    """Definition des Trainingsprozess für einen einzelnen Worker"""
    print(f"Train worker process using GPU: {gpu} for training", flush=True)
    torch.cuda.set_device(gpu) #  GPU wird festgelegt

    pad_idx = vocab_tgt["<blank>"] # Padding
    d_model = 512 # Dimension Embedding
    model = make_model(len(vocab_src), len(vocab_tgt), N=6) # Erstellen des Transformers
    model.cuda(gpu) # Festlegen GPU wird festgelegt
    module = model
    is_main_process = True
    if is_distributed:
        dist.init_process_group(
            "nccl", init_method="env://", rank=gpu, world_size=ngpus_per_node
        )
        model = DDP(model, device_ids=[gpu])
        module = model.module
        is_main_process = gpu == 0

    criterion = LabelSmoothing( # Label Smoothing wird initialisiert
        size=len(vocab_tgt), padding_idx=pad_idx, smoothing=0.1
    )
    criterion.cuda(gpu) # auf GPU gelegt

    train_dataloader, valid_dataloader = create_dataloaders( # Trainings- und Validierungsdataloader
        gpu,
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=config["batch_size"] // ngpus_per_node,
        max_padding=config["max_padding"],
        is_distributed=is_distributed,
    )

    optimizer = torch.optim.Adam( # Auswahl Gradientenabstiegsverfahren
        model.parameters(), lr=config["base_lr"], betas=(0.9, 0.98), eps=1e-9
    )
    lr_scheduler = LambdaLR( # Auswahl Lernrate
        optimizer=optimizer,
        lr_lambda=lambda step: rate(
            step, d_model, factor=1, warmup=config["warmup"]
        ),
    )
    train_state = TrainState() # Zustand des Trainingsvorgangs

    for epoch in range(config["num_epochs"]):
        if is_distributed: # Verteiltes Lernen:
            train_dataloader.sampler.set_epoch(epoch)
            valid_dataloader.sampler.set_epoch(epoch)

        model.train() # Nicht verteiltes Lernen:
        print(f"[GPU{gpu}] Epoch {epoch} Training ====", flush=True)
        _, train_state = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in train_dataloader),
            model,
            SimpleLossCompute(module.generator, criterion),
            optimizer,
            lr_scheduler,
            mode="train+log",
            accum_iter=config["accum_iter"],
            train_state=train_state,
        )

        GPUtil.showUtilization() # GPU Auslastung anzeigen
        if is_main_process:
            file_path = "%s%.2d.pt" % (config["file_prefix"], epoch)
            torch.save(module.state_dict(), file_path)
        torch.cuda.empty_cache()

        print(f"[GPU{gpu}] Epoch {epoch} Validation ====", flush=True)
        model.eval()
        sloss = run_epoch(
            (Batch(b[0], b[1], pad_idx) for b in valid_dataloader),
            model,
            SimpleLossCompute(module.generator, criterion),
            DummyOptimizer(),
            DummyScheduler(),
            mode="eval",
        )
        print(sloss) # Verlust anzeigen
        torch.cuda.empty_cache()

    if is_main_process:
        file_path = "%sfinal.pt" % config["file_prefix"]
        torch.save(module.state_dict(), file_path) # Modell speichern


def train_distributed_model(vocab_src, vocab_tgt, spacy_de, spacy_en, config):
    """Verteiles Lernen"""
    ngpus = torch.cuda.device_count()
    os.environ["MASTER_ADDR"] = "localhost"
    os.environ["MASTER_PORT"] = "12356"
    print(f"Number of GPUs detected: {ngpus}")
    print("Spawning training processes ...")
    mp.spawn(
        train_worker,
        nprocs=ngpus,
        args=(ngpus, vocab_src, vocab_tgt, spacy_de, spacy_en, config, True),
    )


def train_model(vocab_src, vocab_tgt, spacy_de, spacy_en, config):
    """Prüft ob parallelisiert trainiert werden soll"""
    if config["distributed"]:
        train_distributed_model(
            vocab_src, vocab_tgt, spacy_de, spacy_en, config
        )
    else:
        train_worker(
            0, 1, vocab_src, vocab_tgt, spacy_de, spacy_en, config, False
        )


def load_trained_model():
    """Erstellt bzw. lädt das trainierte Modell"""
    config = {
        "batch_size": 32,
        "distributed": False,
        "num_epochs": 8,
        "accum_iter": 10,
        "base_lr": 1.0,
        "max_padding": 72,
        "warmup": 3000,
        "file_prefix": "multi30k_model_",
    }
    model_path = "multi30k_model_final.pt" # rel. Pfad für das Modell
    if not exists(model_path):
        train_model(vocab_src, vocab_tgt, spacy_de, spacy_en, config)

    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.load_state_dict(torch.load("multi30k_model_final.pt"))
    return model

model = load_trained_model()

Finished.
Vocabulary sizes:
8315
6384


# Part 3: Transformer Auswerten

## Beispiel Sätze

In [25]:
def greedy_decode(model, src, src_mask, max_len, start_symbol):
    """Nehme Token mit höchster Wahrscheinlichkeit in jedem Schritt"""
    memory = model.encode(src, src_mask)
    ys = torch.zeros(1, 1).fill_(start_symbol).type_as(src.data)
    for i in range(max_len - 1):
        out = model.decode(
            memory, src_mask, ys, subsequent_mask(ys.size(1)).type_as(src.data)
        )
        prob = model.generator(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.data[0]
        ys = torch.cat(
            [ys, torch.zeros(1, 1).type_as(src.data).fill_(next_word)], dim=1
        )
    return ys


def check_outputs(
    valid_dataloader,
    model,
    vocab_src,
    vocab_tgt,
    n_examples=15,
    pad_idx=2,
    eos_string="</s>",
):
    results = [()] * n_examples
    for idx in range(n_examples):
        print("\nExample %d ========\n" % idx)
        b = next(iter(valid_dataloader))
        rb = Batch(b[0], b[1], pad_idx)
        greedy_decode(model, rb.src, rb.src_mask, 64, 0)[0]

        src_tokens = [
            vocab_src.get_itos()[x] for x in rb.src[0] if x != pad_idx
        ]
        tgt_tokens = [
            vocab_tgt.get_itos()[x] for x in rb.tgt[0] if x != pad_idx
        ]

        print(
            "Source Text (Input)        : "
            + " ".join(src_tokens).replace("\n", "")
        )
        print(
            "Target Text (Ground Truth) : "
            + " ".join(tgt_tokens).replace("\n", "")
        )
        model_out = greedy_decode(model, rb.src, rb.src_mask, 72, 0)[0]
        model_txt = (
            " ".join(
                [vocab_tgt.get_itos()[x] for x in model_out if x != pad_idx]
            ).split(eos_string, 1)[0]
            + eos_string
        )
        print("Model Output               : " + model_txt.replace("\n", ""))
        results[idx] = (rb, src_tokens, tgt_tokens, model_out, model_txt)
    return results


def run_model_example(n_examples=5):
    global vocab_src, vocab_tgt, spacy_de, spacy_en

    print("Preparing Data ...")
    _, valid_dataloader = create_dataloaders(
        torch.device("cpu"),
        vocab_src,
        vocab_tgt,
        spacy_de,
        spacy_en,
        batch_size=1,
        is_distributed=False,
    )

    print("Loading Trained Model ...")

    model = make_model(len(vocab_src), len(vocab_tgt), N=6)
    model.load_state_dict(
        torch.load("multi30k_model_final.pt", map_location=torch.device("cpu"))
    )

    print("Checking Model Outputs:")
    example_data = check_outputs(
        valid_dataloader, model, vocab_src, vocab_tgt, n_examples=n_examples
    )
    return model, example_data


execute_example(run_model_example)

Preparing Data ...
Loading Trained Model ...
Checking Model Outputs:

Example 0 ========

Source Text (Input)        : <s> Eine Frau in einem blauen T-Shirt spielt ein Instrument . </s>
Target Text (Ground Truth) : <s> A woman in a blue shirt is playing an instrument . </s>
Model Output               : <s> A woman in a blue shirt is playing an instrument . </s>

Example 1 ========

Source Text (Input)        : <s> Eine Person beim Parasailing über einem großen Gewässer . </s>
Target Text (Ground Truth) : <s> A person is parasailing over a large body of water . </s>
Model Output               : <s> A person is parasailing over a large body of water . </s>

Example 2 ========

Source Text (Input)        : <s> Viele Stühle sind leer und nur einige wenige Menschen genießen die Sonne . </s>
Target Text (Ground Truth) : <s> Many of the chairs are empty with only a few people enjoying the sun . </s>
Model Output               : <s> Many people are in the same time and some people enjoy the su